# Data Literacy Project
#### University of Tübingen, Winter Term 2021/22

### Testing 01
---

In [1]:
import requests
import json
import pandas as pd


In [2]:
apiResponse = requests.get(
    'https://de.openparliament.tv/api/v1/electoralPeriod/DE-019')
data = json.loads(apiResponse.text)
data = pd.json_normalize(data, ['data', 'relationships', 'sessions', 'data'])
data


,data.type,data.id,data.attributes.dateStart,data.attributes.dateEnd,data.attributes.number,data.attributes.agendaItemCount,data.links.self
0,session,DE-0190001,2017-10-24T11:00,2017-10-24T17:03,1,9,https://de.openparliament.tv/api/v1/session/DE...
1,session,DE-0190002,2017-11-21T10:00,2017-11-21T19:42,2,15,https://de.openparliament.tv/api/v1/session/DE...
2,session,DE-0190003,2017-11-22T12:30,2017-11-22T17:31,3,9,https://de.openparliament.tv/api/v1/session/DE...
3,session,DE-0190004,2017-12-12T09:00,2017-12-12T18:01,4,13,https://de.openparliament.tv/api/v1/session/DE...
4,session,DE-0190005,2017-12-13T11:00,2017-12-13T19:46,5,16,https://de.openparliament.tv/api/v1/session/DE...
...,...,...,...,...,...,...,...
234,session,DE-0190235,2021-06-23T12:00,2021-06-23T21:24,235,10,https://de.openparliament.tv/api/v1/session/DE...
235,session,DE-0190236,2021-06-24T9:00,2021-06-25T2:30,236,30,https://de.openparliament.tv/api/v1/session/DE...
236,session,DE-0190237,2021-06-25T9:00,2021-06-25T18:16,237,12,https://de.openparliament.tv/api/v1/session/DE...
237,session,DE-0190238,2021-08-25T12:00,2021-08-25T19:33,238,5,https://de.openparliament.tv/api/v1/session/DE...


In [3]:
def getDictOfTextContents(textContents):
    body = textContents[0]['textBody']
    subBody = []
    for j in range(len(body)):
        textPart = body[j]
        subBody.append({textPart['type']:
                        ' '.join([s['text'] for s in textPart['sentences']])})
    return subBody


In [4]:
base_request = 'https://de.openparliament.tv/api/v1/search/media/?'
max_page = 5

apiResponse = requests.get(base_request + 'page=' + str(1)).json()
result = apiResponse['data']
for page in range(2, max_page+1):
    apiResponse = requests.get(base_request + 'page=' + str(page)).json()
    result.extend(apiResponse['data'])

data = pd.json_normalize(result)
# data to store
# - relationships.people.data.attributes.label                           (main-speaker)
# - relationships.people.data.attributes.party.label                     (main-speaker party)
# - attributes.TextContent.DICT(textBody.type, textBody.sentences.sentences.text) (speech and comments)


In [5]:
dataExtracted = data[['attributes.textContents', 'relationships.people.data']]

dataExtracted['sentences'] = dataExtracted.apply(
    lambda dataRow: getDictOfTextContents(dataRow['attributes.textContents']), axis=1)
dataExtracted = dataExtracted.drop(columns=['attributes.textContents'])

dataExtracted['main-speaker'] = dataExtracted.apply(
    lambda dataRow: dataRow['relationships.people.data'][0]['attributes']['label'], axis=1)
dataExtracted['main-speaker-party'] = dataExtracted.apply(
    lambda dataRow: dataRow['relationships.people.data'][0]['attributes']['party']['label'], axis=1)
dataExtracted = dataExtracted.drop(columns=['relationships.people.data'])


dataExtracted.loc[1].sentences
dataExtracted


/tmp/ipykernel_47293/3676494636.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataExtracted['sentences'] = dataExtracted.apply(


,sentences,main-speaker,main-speaker-party
0,"[{'speech': 'Guten Morgen, liebe Kolleginnen u...",Hermann Otto Solms,Freie Demokratische Partei
1,[{'speech': 'Herr Präsident! Liebe Kolleginnen...,Jan Korte,Die Linke
2,[{'speech': 'Herr Präsident! Sehr verehrte Gäs...,Michael Grosse-Brömer,Christlich Demokratische Union Deutschlands
3,[{'speech': 'Sehr geehrter Herr Präsident! Mei...,Britta Haßelmann,Bündnis 90/Die Grünen
4,"[{'speech': 'Wir kommen nun zur Abstimmung.'},...",Hermann Otto Solms,Freie Demokratische Partei
5,[{'speech': 'Ich rufe den Tagesordnungspunkt 5...,Wolfgang Schäuble,Christlich Demokratische Union Deutschlands
6,[{'speech': 'a) Beratung des Antrags der Frakt...,Wolfgang Schäuble,Christlich Demokratische Union Deutschlands
7,[{'speech': 'Sehr geehrter Herr Präsident! Lie...,Michael Grosse-Brömer,Christlich Demokratische Union Deutschlands
8,[{'speech': 'Sehr geehrter Herr Präsident! Seh...,Jan Nolte,Alternative für Deutschland
9,[{'speech': 'Herr Präsident! Liebe Kolleginnen...,Tobias Lindner,Bündnis 90/Die Grünen


In [6]:
# Store data for use in other notebooks. 
%store dataExtracted

Stored 'dataExtracted' (DataFrame)
